# Sentinel 2 Dynamic World Metrics
v20240626 and v20240709

Initial exploratory analysis of Dynamic World-based metrics for Susitna Wolf project, completed in 2024.

The Dynamic World V1 product (https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1) is generated for all Sentinel 2 granules with <35% cloud cover. It is a 10 m resolution product with 9 classes. It can be summarized to provide continuous and categorical products that characterize the snow phenology and land cover dynamics at fine spatial resolution.

## Study Area
The mapped area was the 20 100-km AKVEG tiles that intersect the Susitna Wolf study area in south-central Alaska. Each tile is 10000x10000 pixels at 10 m resolution. The tiles were clipped by the overall AKVEG map extent polygon, which has the effect of masking some ocean that is not adjacent to the coastline.


## Snow Phenology
GEE script snapshot: https://code.earthengine.google.com/426d422716a367f7684ad215af1a489e

GEE script path: https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Adynamic_world_metrics%2Fs2_dynamic_world_snow_metrics_v20240626

The Dynamic World product provides a time-series of snow and non-snow observations for each 10m pixel for the time period 2016–present. We summarized the Dynamic World product to characterize the snow cover regime for the study area. The spring snow cover regime is defined by the normal date when the snow-covered season ends and the snow-free season starts. The fall snow cover regime is defined by the normal date when the snow-free season ends and the snow-covered season starts.

### Filtering and Daily Mosaic
Only a spatial filtering step was applied, to select the dynamic world granules that intersected each AKVEG tile. 

Dynamic World products have a cloud and cloud-shadow mask applied, and no additional quality masking was performed.

The data were mosaicked based on date and orbit number, to flatten data from overlap areas that occur at the edges of MGRS tiles. Without this step, there would be duplicate data from the overlap areas.

### Moving window modal land cover
For each day of year between January 15–December 16 (DOY 15–350), the modal land cover was calculated using a time window of 29 days, including 14 days before and after the target day of year. Data from all years was combined, and the year was not considered in the analysis. We did not calculate modal land cover for the dates between December 17-January 14 due to the complexity of the transition from December 31 to January 1. In general it is unlikely that there is land where the transition date from snow-free to snow-covered occurs near January 1, though this is a normal date for some snow-covered lake or sea ice.

Dynamic World does include class probabilities and we experimented with using that data in the summaries, but it was challenging to define a consistent threshold.

TODO: Consider reclassifying to snow/not-snow and calculating mode on binary snow/non-snow rather than on all 9 classes.

### Snow transition dates
The mode-summarized time-series was simplified to a binary snow/non-snow classification.

It was then split into two separate time-series, which were then analyzed separately: a snow-cover depletion time-series (January 15–August 15), and a snow-cover accumulation time-series (August 15–December 16). For the snow-cover depletion period, the maximum DOY where the modal class was snow ('last_snow_doy') and the minimum DOY where the modal class was snow-free ('first_snowfree_doy') were extracted. If none of the dates had a modal class of snow, the last snow DOY was set to zero, and if none of the dates had a snow-free modal class, the first snow-free DOY was set to 351. 

Similarly, for the snow-cover accumulation period, the maximum DOY where the modal class was snow-free ('last_snowfree_doy') and the minimum DOY where the modal class was snow ('first_snow_doy') was extracted. If none of the dates had a snow-free modal class, the last snow-free DOY was set to zero, and if none of the dates had a snow modal class, the first snow DOY was set to 351.

For each transition DOY, the sample size of observations in the 29-day window corresponding the transition date was saved in additional bands. This can be used as an indicator of whether there was sufficient data to characterize the transition DOY. The total number of observations in the entire time-series was also saved.

Saved as asset in ee.imageCollection('projects/akveg-map/assets/dynamic_world_metrics/s2_snow_doys_from_dw_227_v20240626')

### Summarization
If the 'last_snow_doy' or 'first_doy_doy' was equal to 15 August (doy 227), the snow regime was set to permanent snow. Similarly, if the 'first_snowfree_doy' or 'last_snowfree_doy' was equal to January 15 (doy 15), the snow regime wsa set to  permanent snow-free.

For the remainder, we calculated the difference between the first snow-free DOY and the last snow DOY ('early_doy_diff'); and the difference between the first snow DOY and the last snow-free DOY ('late_doy_diff'). In cases where there was a simple, binary break between the snow and non-snow seasons, the difference would equal +1 (e.g. the first snow-free DOY was one day after the last snow DOY in the spring). For values other than +1, the snow transition was more complicated, implying multiple transitions between snow and snow-free.

If the 'last_snow_doy' and 'first_snowfree_doy' were within 7 days of each other (i.e. absolute value of 'early_doy_diff' <= 7), the final 'snowfree_doy' was set to the mean of the two dates. Otherwise, the 'last_snow_doy' was used, because the 'first_snowfree_doy' was observed to be noisier, often reflecting a period with spurious snow-free observations during deep winter.

Similarly, 'snow_doy' was assigned as the mean of 'last_snowfree_doy' and 'first_snow_doy' if the dates were within 7 days of each other. Otherwise, the 'first_snow_doy' was used, because 'last_snowfree_doy' tended to reflect noisy deep winter dates in cases where the two dates were further apart.

Saved as ee.Image('projects/akveg-map/assets/dynamic_world_metrics/s2_snow_doys_parsed_v20240626b')
 

## Land Cover Frequency Metrics
GEE script snapshot: https://code.earthengine.google.com/338b8a3f4ea0f0e0f05d3b2c61947b6f

GEE script path: https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Adynamic_world_metrics%2Fs2_dynamic_world_lc_metrics

### Land Cover Frequency in Non-snow and Snow seasons
We calculated the count of each Dynamic World land cover class separately for the snow season (doy > 'snowfree_doy' and doy <= 'snow_doy' and snow-free season (doy > 'snow_doy' and doy <= 'snowfree_doy'), splitting the seasons using the summarized results of the snow phenology analysis above. The rationale for splitting the seasons was that some of the non-snow observations during the snow season are likely to be spurious.

Similar to the snow analysis, no filtering or additional masking was applied. The data were mosaicked based on date and orbit number, to flatten data from overlap areas that occur at the edges of MGRS tiles. 

In addition to calculating the counts for each land cover class, the modal land cover class was extracted for each season.

### Water Frequency Metrics
Water frequency was calculated separately based on both the 'water' and 'flooded_vegetation' classes. Water frequency was extracted for monthly windows (lumping across years), covering the months May–September. The water metrics were not split based on the snow seasons. For each month, the count of 'water', 'flooded_vegetation', 'nonwater', and 'snow_and_ice' observations was tallied. The 'nonwater' class did not include 'snow_and_ice' because it is ambiguous from the label whether the observation is associated with a temporary or permanent water body.

Water frequency was calculated as the count of 'water' divided by the count of all classes except 'snow_and_ice'. Flooded vegetation frequency was calculated in a similar way. 

Finally, summer (June/July/August) water frequency and flooded vegetation frequency was calculated as the mean of the values for those three months (i.e. not weighted by the number of observations in each month).